1. fai una join su id
2. prima cosa dividi tra camel e kebab (tieni traccia di chi sta rispondendo)
3. dividi per cs background 
4. calcola accuracy (numeri di risposte corrette / il numero di risposte totali) per ogni persona di ogni gruppo e poi fai la media intra gruppo : cosi ho l'accuracy media
e poi lo fai per ogni cs background
5. tenendo conto di solo le risposte corrette, calcoli il tempo medio di risposta di ogni persona di ogni gruppo
6. poi fai il paired t test per vedere se ci sono differenze significative tra i gruppi

In [16]:
import pandas as pd
import numpy as np

# Load the data
df = pd.read_csv('code-comprehension.runs.csv')

# Loop through columns containing 'questionId' and apply the mapping
for i in range(20):  # Iterate over the 20 questionId columns
    column_name = f'results[{i}].questionId'  # Dynamically format the column name
    if column_name in df.columns:  # Ensure the column exists
        # Map the values based on the conditions
        df[column_name] = np.where(
            df[column_name].between(0, 9),  # Condition for camelCase
            'camelCase',
            np.where(
                df[column_name].between(10, 19),  # Condition for kebab-case
                'kebab-case',
                df[column_name]  # Keep the original value if not in range
            )
        )

# Save the updated data back to CSV
df.to_csv('code-comprehension.runs.csv', index=False)

In [18]:
def transform_user_data(file_path, output_path):
    # Load the dataset
    data = pd.read_csv(file_path)

    # Identify user-specific and demographic columns
    user_cols = ['_id', 'demographics.age', 'demographics.csBackground',
                 'demographics.yearsOfExperience']
    question_cols = [col for col in data.columns if col.startswith('results')]

    # Separate the data for reshaping
    user_data = data[user_cols]
    question_data = data[question_cols]

    # Melt the question-related data to long format
    melted = question_data.melt(ignore_index=False, var_name='question_metric', value_name='value')

    # Extract question number and metric type (elapsedTime, isCorrect, questionId)
    melted['question'] = melted['question_metric'].str.extract(r'results\[(\d+)\]')
    melted['metric'] = melted['question_metric'].str.extract(r'results\[\d+\]\.(.+)')

    # Pivot to have one row per question with columns for elapsedTime, isCorrect, questionId
    reshaped = melted.pivot_table(index=[melted.index, 'question'], columns='metric', values='value', aggfunc='first').reset_index()

    # Merge back with user demographic data
    reshaped = reshaped.merge(user_data, left_on='level_0', right_index=True).drop(columns=['level_0'])

    # Rename columns for clarity
    reshaped = reshaped.rename(columns={
        'question': 'question_number',
        'elapsedTime': 'elapsed_time',
        'isCorrect': 'is_correct',
        'questionId': 'question_id'
    })

    reshaped['question_id'] = reshaped['question_id'].replace(20, 'camelCase')

    # Convert data types where necessary
    reshaped['question_number'] = reshaped['question_number'].astype(int)
    reshaped['elapsed_time'] = pd.to_numeric(reshaped['elapsed_time'], errors='coerce')
    reshaped['is_correct'] = reshaped['is_correct'].astype('bool', errors='ignore')

    # print(reshaped.head(20))

    # Save the transformed data to a CSV file
    reshaped.to_csv(output_path, index=False)
    
transform_user_data('code-comprehension.runs.csv', 'transformed_data.csv')

In [ ]:
import pandas as pd
# Open transformed data
transformed_data = pd.read_csv('results/transformed_data.csv')

transformed_data = transformed_data[transformed_data['is_correct'] == True]

transformed_data.to_csv('correct_answers.csv', index=False)


In [16]:
import pandas as pd
# Open transformed data
transformed_data = pd.read_csv('results/transformed_data.csv')

# Filter rows for each case style
kebab_case = transformed_data[transformed_data['question_id'] == 'kebab-case']
camel_case = transformed_data[transformed_data['question_id'] == 'camelCase']

# Filter rows where is_correct is True
kebab_case = kebab_case[kebab_case['is_correct'] == True]
camel_case = camel_case[camel_case['is_correct'] == True]

# Combine the rows for each `_id` into two columns
paired_data = pd.DataFrame()
for unique_id in transformed_data['_id'].unique():
    kebab_subset = kebab_case[kebab_case['_id'] == unique_id].head(10).reset_index(drop=True)
    camel_subset = camel_case[camel_case['_id'] == unique_id].head(10).reset_index(drop=True)
    paired_subset = pd.concat([kebab_subset[['elapsed_time']].rename(columns={'elapsed_time': 'kebab-case'}),
                                camel_subset[['elapsed_time']].rename(columns={'elapsed_time': 'camelCase'})], axis=1)
    paired_subset['_id'] = unique_id
    paired_data = pd.concat([paired_data, paired_subset], ignore_index=True)

paired_data.to_csv('paired_dataTEST.csv', index=False)

In [21]:
#open transofrmed data
transformed_data = pd.read_csv('results/transformed_data.csv')

print(transformed_data.shape[0]/20)

36.0


In [20]:
import pandas as pd
# Open transformed data
transformed_data = pd.read_csv('results/transformed_data.csv')

csBackground = transformed_data[transformed_data['demographics.csBackground'] == True]

# transformed_data.to_csv('csBackground_answers.csv', index=False)

correct = csBackground[csBackground['is_correct'] == True].shape[0]
incorrect = csBackground[csBackground['is_correct'] == False].shape[0]
ratio = correct / (correct + incorrect)

print('CSBG correct: ', correct, 'incorrect: ', incorrect, 'ratio: ', ratio)

nonCsBackground_answers = transformed_data[transformed_data['demographics.csBackground'] == False]
correct = nonCsBackground_answers[nonCsBackground_answers['is_correct'] == True].shape[0]
incorrect = nonCsBackground_answers[nonCsBackground_answers['is_correct'] == False].shape[0]
ratio = correct / (correct + incorrect)

# nonCsBackground_answers.to_csv('nonCsBackground_answers.csv', index=False)

print('NON CSBG correct: ', correct, 'incorrect: ', incorrect, 'ratio: ', ratio)


CSBG correct:  457 incorrect:  43 ratio:  0.914
NON CSBG correct:  207 incorrect:  13 ratio:  0.9409090909090909
